In [2]:
!pip install -qU \
  langchain==0.0.162 \
  openai==0.27.7 \
  tiktoken==0.4.0 \
  "pinecone-client[grpc]"==2.2.2

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 3.4.1 requires typer<0.5.0,>=0.3.0, but you have typer 0.9.0 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [3]:
import tiktoken

tiktoken.encoding_for_model('gpt-3.5-turbo')
tokenizer = tiktoken.get_encoding('cl100k_base')

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

In [31]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=20,
    length_function=tiktoken_len,
    separators=["\n\n", "\n", " ", ""]
)

In [5]:
import os

# get openai api key from platform.openai.com
OPENAI_API_KEY = "[OPENAI_API_KEY]"

In [7]:
from langchain.embeddings.openai import OpenAIEmbeddings

model_name = 'text-embedding-ada-002'

embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=OPENAI_API_KEY
)

In [8]:
text_file_path = "/Users/aditya_rathod/Desktop/Courses/Fall23/Capstone/python-backend/kb.txt"
file_text= open(text_file_path, "r").read()
chunks = text_splitter.split_text(file_text)
print(len(chunks))

4


In [9]:
from langchain.embeddings.openai import OpenAIEmbeddings

model_name = 'text-embedding-ada-002'

embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=OPENAI_API_KEY
)

In [10]:
res = embed.embed_documents(chunks)
len(res), len(res[0])

(4, 1536)

In [11]:
index_name = 'capstone-langchain-retrieval-augmentation'

In [32]:
import pinecone

# find API key in console at app.pinecone.io
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY') or 'PINECONE_API_KEY'
# find ENV (cloud region) next to API key in console
PINECONE_ENVIRONMENT = os.getenv('PINECONE_ENVIRONMENT') or 'PINECONE_ENVIRONMENT'

pinecone.init(
    api_key="[pinecone_api_key]",
    environment="[pinecone_environment]"
)

if index_name not in pinecone.list_indexes():
    # we create a new index
    pinecone.create_index(
        name=index_name,
        metric='cosine',
        dimension=len(res[0])  # 1536 dim of text-embedding-ada-002
    )

In [33]:
index = pinecone.GRPCIndex(index_name)

index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 4}},
 'total_vector_count': 4}

In [34]:
from tqdm.auto import tqdm
from uuid import uuid4

batch_limit = 100

texts = []
metadatas = []
data= [open("kb.txt", "r").read()]
for i, record in enumerate(tqdm(data)):
    # first get metadata fields for this record
    # metadata = {
    #     'wiki-id': str(record['id']),
    #     'source': record['url'],
    #     'title': record['title']
    # }
    # now we create chunks from the record text
    record_texts = text_splitter.split_text(record)
    # create individual metadata dicts for each chunk
    record_metadatas = [{
        "chunk": j, "text": record
    } for j, text in enumerate(record_texts)]
    # append these to current batches
    texts.extend(record_texts)
    metadatas.extend(record_metadatas)
    # if we have reached the batch_limit we can add texts
    if len(texts) >= batch_limit:
        ids = [str(uuid4()) for _ in range(len(texts))]
        embeds = embed.embed_documents(texts)
        index.upsert(vectors=zip(ids, embeds, metadatas))
        texts = []
        metadatas = []

if len(texts) > 0:
    ids = [str(uuid4()) for _ in range(len(texts))]
    embeds = embed.embed_documents(texts)
    index.upsert(vectors=zip(ids, embeds, metadatas))

100%|██████████| 1/1 [00:00<00:00, 236.63it/s]


In [35]:
from langchain.vectorstores import Pinecone

text_field = "text"

# switch back to normal index for langchain
index = pinecone.Index(index_name)

vectorstore = Pinecone(
    index, embed.embed_query, text_field
)

In [36]:
query = "What is dynamic programming?"

vectorstore.similarity_search(
    query,  # our search query
    k=1  # return 3 most relevant docs
)

[Document(page_content='What is a Fibonacci Series?\nThe Fibonacci series is a sequence of numbers where each number is the sum of the two preceding ones. It starts from 0 and 1. The sequence goes: 0, 1, 1, 2, 3, 5, 8, 13, 21, and so on.\n\nTo make it even clearer:\n\nThe 1st number in the series is 0.\nThe 2nd number in the series is 1.\nThe 3rd number is the sum of the 1st and 2nd numbers: 0 + 1 = 1.\nThe 4th number is the sum of the 2nd and 3rd numbers: 1 + 1 = 2.\nAnd so on...\nOrigin and Significance\nThe series is named after Leonardo of Pisa, who was known as Fibonacci. He introduced this sequence to Western mathematics in his 1202 book "Liber Abaci." However, it\'s worth noting that the sequence had been previously described in Indian mathematics.\n\nThe Fibonacci sequence has many interesting properties and appears in various domains:\n\nIn nature: The arrangement of leaves on plants, the branching of trees, and the spiral patterns of sunflower seeds often follow the Fibonacci

In [39]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

# completion llm
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name='gpt-3.5-turbo-16k',
    temperature=0.0
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

In [40]:
qa.run(query)

'Dynamic programming (DP) is a method used for solving complex problems by breaking them down into smaller, simpler subproblems. These subproblems are solved and stored for future reference, so if the same problem comes up again, we can simply look up the solution rather than re-solving it from scratch. This is particularly beneficial for optimization problems where you want to find the best possible solution out of many.\n\nThe key concepts in dynamic programming include overlapping subproblems, optimal substructure, memoization, and tabulation. Overlapping subproblems refer to breaking down a problem into smaller subproblems that are not independent and are reused multiple times. Optimal substructure means that the overall optimal solution can be constructed from the optimal solutions of its subproblems. Memoization is the technique of storing solutions to overlapping subproblems to avoid redundant computations. Tabulation is a bottom-up approach where you solve the problem by solvin